In [2]:
pwd
set more off
clear all


C:\Users\wue04\Box Sync\fresh




In [4]:
import delimited raw-output\closest-facility\closest_2012.csv, clear

(8 vars, 32,882 obs)


In [6]:
tab facilityrank


FacilityRan |
          k |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     27,674       84.16       84.16
          2 |      2,987        9.08       93.25
          3 |      1,704        5.18       98.43
          4 |        517        1.57      100.00
------------+-----------------------------------
      Total |     32,882      100.00
